Creating Satellite Image Timelapses

For this demo, I will create a timelapse of satellite images using [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/). 

This code is based of the work from __Milan Janosov__. Check his [article](https://towardsdatascience.com/creating-satellite-image-timelapses-2b479f86ff52) out on this topic where he uses the new ESA Sentinel Hub API.



Getting Started



MPC hosts datasets using the [STAC](https://stacspec.org/en) (SpatioTemporal Asset Catalog) specification.

 To interact with these datasets, we need to install the [pystac-client](https://pypi.org/project/pystac-client/) and [Planetary Computer SDK](https://pypi.org/project/planetary-computer/) via __pip__

In [6]:
import sys
!{sys.executable} -m pip install --quiet pystac-client planetary-computer --upgrade 

Next, we need to create a `pystac_client.Client` to access the data we want to pull from the MPC

In [7]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

Searching

Now we can use the STAC APT to search for our assets that meet a certian criteria. This criteria could be certain dates and the area we want to anaylze.

Here, I will look at Cueno, a city in Piedmont, Italy during May 2023.

F1 Fans: This city is just north of Monte Carlo 

In [9]:

time_range = "2023-05-01/2023-05-31"
bbox = [6.368637,43.492783,7.673950,44.200913]

search = catalog.search(collections=["landsat-c2-l2"], bbox=bbox, datetime=time_range)
items = search.get_all_items()
len(items)

28